In [1]:
!pip install -r requirements.txt
!pip install more_itertools

  Using cached numpy-1.19.2-cp38-cp38-manylinux2010_x86_64.whl (14.5 MB)
  Using cached oauth2client-4.1.3-py2.py3-none-any.whl (98 kB)
  Using cached pandas-1.1.3-cp38-cp38-manylinux1_x86_64.whl (9.3 MB)
  Using cached requests-2.25.0-py2.py3-none-any.whl (61 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.2.0
    Uninstalling pandas-1.2.0:
      Successfully uninstalled pandas-1.2.0
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before

In [2]:
from more_itertools import unique_everseen
from scipy.optimize import linprog as lp
import fooddatacentral as fdc
import pandas as pd
import numpy as np

apikey = "ogIXF0LL4bt0Nyo7Yrq09PvDyeAxZNYBvGklse01"
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
# 1.2 Dietary Reference Intakes 

diet_minimums = pd.read_csv('diet_minimums.csv').drop(columns=['Unnamed: 0', 'Source']).set_index('Nutrition')
diet_maximums = pd.read_csv('diet_maximums.csv').drop(columns=['Unnamed: 0', 'Source']).set_index('Nutrition')

def get_age_column(age):
    global diet_minimums
    age_ranges = []
    for i in diet_minimums.columns:
        age_ranges += [i.split(' ')[1]]
    age_ranges = list(unique_everseen(age_ranges))[:-1]
    
    for ranges in age_ranges:
        results = list(map(int, ranges.split('-')))
        if age in range(results[0], results[1] + 1): return '{0}-{1}'.format(results[0], results[1])
    return '51+'

def get_dietary_reqs(age, gender):
    if (age <= 0):
        raise Exception("No ages below 0")
    
    global diet_minimums
    target_column_name = 'C 1-3' if age in range(1, 4) else '{0} {1}'.format(gender, get_age_column(age))
    return diet_minimums[target_column_name]

def get_dietary_max(age, gender):
    if (age <= 0):
        raise Exception("No ages below 0")
        
    global diet_maximums
    target_column_name = 'C 1-3' if age in range(1, 4) else '{0} {1}'.format(gender, get_age_column(age))
    return diet_maximums[target_column_name]

def get_dietary_min(age, gender, activity):
    output = get_dietary_reqs(age, gender)
    activity_scale = 1

    if activity == 2:
        activity_scale = 200 / output["Energy"] + 1
    
    elif activity == 3:
        activity_scale = 400 / output["Energy"] + 1
    
    output = output * activity_scale       
    return output

In [6]:
#testing dietary in function
get_dietary_min(23, 'M', 3)

Nutrition
Energy                           2800.000
Protein                            65.333
Fiber, total dietary               39.200
Folate, DFE                       466.667
Calcium, Ca                      1166.667
Carbohydrate, by difference       151.667
Iron, Fe                            9.333
Magnesium, Mg                     466.667
Niacin                             18.667
Phosphorus, P                     816.667
Potassium, K                     5483.333
Riboflavin                          1.517
Thiamin                             1.400
Vitamin A, RAE                   1050.000
Vitamin B-12                        2.800
Vitamin B-6                         1.517
Vitamin C, total ascorbic acid    105.000
Vitamin E (alpha-tocopherol)       17.500
Vitamin K (phylloquinone)         140.000
Zinc, Zn                           12.833
Name: M 19-30, dtype: float64

In [8]:
#code for people who may not have Json
!gpg -d --batch --passphrase "noodle octopus" ../students-9093fa174318.json.gpg > ../students-9093fa174318.json

gpg: AES256 encrypted data
gpg: encrypted with 1 passphrase


In [53]:
#Aggregating Foods into one Data Frame
from eep153_tools import read_sheets

Omnivore_price = read_sheets("1M_bUswM_uKnkntd3D-OIcm9x_zuQW17oXikwIl8uh8I",
                 sheet="Omnivore",
                 json_creds='../students-9093fa174318.json')

Omnivore_price.drop(columns=["Notes"], inplace=True)

Omnivore_price['FDC Quantity'] = Omnivore_price[['Quantity', 'Units']].T.apply(lambda x : fdc.units(x['Quantity'], x['Units']))
Omnivore_price['FDC Price'] = Omnivore_price['Price'] / Omnivore_price['FDC Quantity']
Omnivore_price.dropna(how='any', inplace=True)
prices = Omnivore_price.groupby('Food', sort = False)['FDC Price'].min()

In [54]:
Omnivore_price

,Food,Quantity,Units,Price,Date,Location,FDC,FDC Quantity,FDC Price
0,Chicken Breast,3.000,lbs,12.870,3/1/21,"Safeway, Berkeley",1309639,13.6077711 hectogram,0.9457831047731247 / hectogram
1,Beef,3.500,lbs,19.220,3/1/21,"Safeway, Berkeley",1271778,15.875732950000002 hectogram,1.2106527654838133 / hectogram
2,Liver (pork),2.490,lbs,7.470,3/1/21,"Safeway, Berkeley",167862,11.294450013000002 hectogram,0.6613867865546326 / hectogram
3,Bacon,16.000,oz,7.390,3/1/21,"Safeway, Berkeley",1272081,4.535923700000001 hectogram,1.629216117546245 / hectogram
4,Rice,5.000,lbs,6.990,3/1/21,"Safeway, Berkeley",532006,22.679618500000004 hectogram,0.30820624253445883 / hectogram
5,Spaghetti,16.000,oz,1.990,3/1/21,"Safeway, Berkeley",1189584,4.535923700000001 hectogram,0.4387199017479063 / hectogram
6,Milk,1.000,gallon,4.490,3/1/21,"Safeway, Berkeley",553224,37.85411783999999 deciliter,0.11861325150880869 / deciliter
7,Cereal,19.500,oz,5.990,3/1/21,"Safeway, Berkeley",1164885,5.5281570093750005 hectogram,1.0835437542460855 / hectogram
8,Broccoli,32.000,oz,2.990,3/1/21,"Safeway, Berkeley",1439817,9.071847400000001 hectogram,0.329591081966392 / hectogram
9,Spinach,16.000,oz,5.000,3/1/21,"Safeway, Berkeley",1197933,4.535923700000001 hectogram,1.1023113109243878 / hectogram


In [55]:
#Although it says 26, it will go into the range of 19-30 for Male and Female
diet_min_female = get_dietary_min(26, 'F', 3)
diet_max_female = get_dietary_max(26, 'F')
diet_min_male = get_dietary_min(26, 'M', 3)
diet_max_male = get_dietary_max(26, 'M')

In [56]:
#Aggregating Nutrient Data into a Dataframe
Omnivore_nutrients = {}
count = 0
for food in Omnivore_price.Food.tolist():
    try:
        FDC = Omnivore_price.loc[Omnivore_price.Food==food, :].FDC[count]
        count +=1
        Omnivore_nutrients[food] = fdc.nutrients(apikey, FDC).Quantity
    except AttributeError: 
        warnings.warn("Couldn't find FDC Code %s for food %s." % (food, FDC))        

food_nutrients = pd.DataFrame(Omnivore_nutrients, dtype=float)

In [57]:
#Dietary Minimum for Female
food_nutrients.loc[diet_min_female.index]

,Chicken Breast,Beef,Liver (pork),Bacon,Rice,Spaghetti,Milk,Cereal,Broccoli,Spinach,...,Pinto Beans,Peanut Butter,Quinoa,Dairy-free Cheese,Soy Milk,Avocados,Apples,Orange,Oatmeal,Hummus
Nutrition,,,,,,,,,,,,,,,,,,,,,
Energy,143.000,179.000,561.000,458.000,367.000,321.000,62.000,378.000,24.000,24.000,...,229.000,597.000,107.000,300.000,172.000,192.000,54.000,47.000,367.000,214.000
Protein,14.290,16.070,21.390,33.330,6.120,14.290,3.330,8.000,2.350,2.350,...,20.000,22.500,2.860,0.000,2.470,0.000,0.410,0.940,11.670,10.710
"Fiber, total dietary",0.000,nan,0.000,0.000,0.000,10.700,0.000,8.000,3.500,2.400,...,34.300,4.800,2.100,0.000,0.400,7.700,2.100,2.400,10.000,7.100
"Folate, DFE",nan,nan,212.000,nan,nan,nan,nan,nan,nan,nan,...,nan,86.000,nan,nan,nan,nan,nan,30.000,nan,nan
"Calcium, Ca",0.000,nan,9.000,0.000,0.000,nan,83.000,351.000,59.000,94.000,...,0.000,49.000,14.000,0.000,123.000,0.000,8.000,40.000,67.000,71.000
"Carbohydrate, by difference",3.570,1.790,2.470,4.170,79.590,69.640,5.000,81.000,4.710,3.530,...,60.000,22.300,20.000,20.000,4.120,0.000,14.050,11.750,63.330,28.570
"Iron, Fe",1.290,1.930,23.300,0.000,0.730,6.430,0.000,10.000,0.820,3.180,...,7.140,1.730,0.770,0.000,0.440,0.000,0.150,0.100,4.500,2.570
"Magnesium, Mg",nan,nan,18.000,nan,nan,143.000,nan,nan,nan,71.000,...,nan,169.000,nan,nan,16.000,nan,nan,10.000,nan,nan
Niacin,nan,nan,15.301,nan,nan,8.929,nan,nan,nan,nan,...,nan,13.300,nan,nan,nan,nan,nan,0.282,nan,nan


In [58]:
#Dietary Minimum for Male
food_nutrients.loc[diet_min_male.index]

,Chicken Breast,Beef,Liver (pork),Bacon,Rice,Spaghetti,Milk,Cereal,Broccoli,Spinach,...,Pinto Beans,Peanut Butter,Quinoa,Dairy-free Cheese,Soy Milk,Avocados,Apples,Orange,Oatmeal,Hummus
Nutrition,,,,,,,,,,,,,,,,,,,,,
Energy,143.000,179.000,561.000,458.000,367.000,321.000,62.000,378.000,24.000,24.000,...,229.000,597.000,107.000,300.000,172.000,192.000,54.000,47.000,367.000,214.000
Protein,14.290,16.070,21.390,33.330,6.120,14.290,3.330,8.000,2.350,2.350,...,20.000,22.500,2.860,0.000,2.470,0.000,0.410,0.940,11.670,10.710
"Fiber, total dietary",0.000,nan,0.000,0.000,0.000,10.700,0.000,8.000,3.500,2.400,...,34.300,4.800,2.100,0.000,0.400,7.700,2.100,2.400,10.000,7.100
"Folate, DFE",nan,nan,212.000,nan,nan,nan,nan,nan,nan,nan,...,nan,86.000,nan,nan,nan,nan,nan,30.000,nan,nan
"Calcium, Ca",0.000,nan,9.000,0.000,0.000,nan,83.000,351.000,59.000,94.000,...,0.000,49.000,14.000,0.000,123.000,0.000,8.000,40.000,67.000,71.000
"Carbohydrate, by difference",3.570,1.790,2.470,4.170,79.590,69.640,5.000,81.000,4.710,3.530,...,60.000,22.300,20.000,20.000,4.120,0.000,14.050,11.750,63.330,28.570
"Iron, Fe",1.290,1.930,23.300,0.000,0.730,6.430,0.000,10.000,0.820,3.180,...,7.140,1.730,0.770,0.000,0.440,0.000,0.150,0.100,4.500,2.570
"Magnesium, Mg",nan,nan,18.000,nan,nan,143.000,nan,nan,nan,71.000,...,nan,169.000,nan,nan,16.000,nan,nan,10.000,nan,nan
Niacin,nan,nan,15.301,nan,nan,8.929,nan,nan,nan,nan,...,nan,13.300,nan,nan,nan,nan,nan,0.282,nan,nan


In [59]:
def solve_subsistence_problem(food_nutrients, prices, diet_min, diet_max, tol=1e-6):    
    c = prices.apply(lambda x:x.magnitude).dropna()
    c = c[list(set(c.index.tolist()).intersection(food_nutrients.columns.tolist()))]
    
    Aall = food_nutrients[c.index].fillna(0)
    Amin = Aall.loc[diet_min.index]
    Amax = Aall.loc[diet_max.index]
    
    A = pd.concat([Amin, -Amax])
    b = pd.concat([diet_min, -diet_max])
    
    result = lp(c, -A, -b, method='interior-point')

    result.A = A
    result.b = b
    result.diet = pd.Series(result.x, index=c.index)

    return result

In [60]:
result_female = solve_subsistence_problem(food_nutrients, prices, diet_min_female, diet_max_female)
result_male = solve_subsistence_problem(food_nutrients, prices, diet_min_male, diet_max_male)

In [61]:
# 1.5 female
daily_cost_female = result_female.fun
print("Cost of diet for female is $%4.2f per day.\n" % (daily_cost_female))

Cost of diet for female is $6.89 per day.



In [62]:
# 1.5 male
daily_cost_male = result_male.fun
print("Cost of diet for male is $%4.2f per day.\n" % (daily_cost_male))

Cost of diet for male is $7.12 per day.



In [63]:
diet = result_male.diet

print("Diet (in 100s of grams or milliliters):")
print(diet[diet >= 1e-6])

Diet (in 100s of grams or milliliters):
Food
Spinach         0.291
Milk            8.246
Pinto Beans     0.736
Spaghetti       0.263
Orange          8.933
Peanut Butter   1.744
Liver (pork)    1.350
dtype: float64


In [64]:
diet = result_female.diet

print("Diet (in 100s of grams or milliliters):")
print(diet[diet >= 1e-6])

Diet (in 100s of grams or milliliters):
Food
Spinach         0.224
Milk            8.313
Pinto Beans     0.819
Orange          9.871
Peanut Butter   1.781
Liver (pork)    0.761
dtype: float64


In [65]:
#female
tab = pd.DataFrame({"Outcome": np.abs(result_female.A).dot(diet), "Recommendation": np.abs(result_female.b)})
print("With the following nutritional outcomes of interest:")
print(tab)

With the following nutritional outcomes of interest:
                                Outcome  Recommendation
Nutrition                                              
Energy                         2662.588        2400.000
Protein                         110.224          55.200
Fiber, total dietary             60.867          33.600
Folate, DFE                     610.697         480.000
Calcium, Ca                    1200.000        1200.000
Carbohydrate, by difference     249.073         156.000
Iron, Fe                         28.369          21.600
Magnesium, Mg                   429.268         372.000
Niacin                           38.119          16.800
Phosphorus, P                   961.178         840.000
Potassium, K                   5640.000        5640.000
Riboflavin                        3.068           1.320
Thiamin                           1.320           1.320
Vitamin A, RAE                 5059.319         840.000
Vitamin B-12                     19.797           2

In [66]:
#male
tab = pd.DataFrame({"Outcome": np.abs(result_male.A).dot(diet), "Recommendation": np.abs(result_male.b)})
print("With the following nutritional outcomes of interest:")
print(tab)

With the following nutritional outcomes of interest:
                                Outcome  Recommendation
Nutrition                                              
Energy                         2662.588        2800.000
Protein                         110.224          65.333
Fiber, total dietary             60.867          39.200
Folate, DFE                     610.697         466.667
Calcium, Ca                    1200.000        1166.667
Carbohydrate, by difference     249.073         151.667
Iron, Fe                         28.369           9.333
Magnesium, Mg                   429.268         466.667
Niacin                           38.119          18.667
Phosphorus, P                   961.178         816.667
Potassium, K                   5640.000        5483.333
Riboflavin                        3.068           1.517
Thiamin                           1.320           1.400
Vitamin A, RAE                 5059.319        1050.000
Vitamin B-12                     19.797           2

In [67]:
print("Constraining nutrients are:")
excess = tab.diff(axis=1).iloc[:, 1]
print(excess.loc[np.abs(excess) < 1e-3].index.tolist())

Constraining nutrients are:
[]


In [71]:
# 1.9
prop_omni_us = .68

# There are 41910 students at UC Berkeley
berkeley_omnis = 41910 * prop_omni_us
berkeley_omnis_cost_female =  round(.5 * (berkeley_omnis * daily_cost_female),2)
print('The total price of berkeley mininum cost diet for female is $' + str(berkeley_omnis_cost_female))

The total price of berkeley mininum cost diet for female is $98149.16


In [72]:
berkeley_omnis_cost_male =  round(.5 * (berkeley_omnis * daily_cost_male),2)
print('The total price of berkeley mininum cost diet for male is $' + str(berkeley_omnis_cost_male))

The total price of berkeley mininum cost diet for male is $101484.5
